# Getting started with InstaNovo

<a target="_blank" href="https://colab.research.google.com/github/instadeepai/InstaNovo/blob/main/notebooks/getting_started_with_instanovo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/instadeepai/InstaNovo/blob/main/notebooks/getting_started_with_instanovo.ipynb">
<img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/> </a>

In this notebook, we demo InstaNovo, a transformer neural network with the ability to translate fragment ion peaks into the sequence of amino acids that make up the studied peptide(s). We evaluate the model on the yeast test fold of nine-species dataset.

![](https://raw.githubusercontent.com/instadeepai/InstaNovo/main/graphical_abstract.jpeg)

**Paper:**

- **De novo peptide sequencing with InstaNovo: Accurate, database-free peptide identification for large scale proteomics experiments** \
  Kevin Eloff, Konstantinos Kalogeropoulos, Oliver Morell, Amandla Mabona, Jakob Berg Jespersen, Wesley Williams, Sam van Beljouw, Marcin Skwark, Andreas Hougaard Laustsen, Stan J. J. Brouns, Anne Ljungars, Erwin M. Schoof, Jeroen Van Goey, Ulrich auf dem Keller, Karim Beguir, Nicolas Lopez Carranza, Timothy P. Jenkins \
  [bioRxiv](https://www.biorxiv.org/content/10.1101/2023.08.30.555055v1), [GitHub](https://github.com/instadeepai/InstaNovo)

**Important:**

It is highly recommended to run this notebook in an environment with access to a GPU. If you are running this notebook in Google Colab:

- In the menu, go to `Runtime > Change Runtime Type > T4 GPU`

## Loading the InstaNovo model

We first install the latest instanovo from PyPi

_Note: this currently installs directly from GitHub, this will be updated in the next release._

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install instanovo

In [ ]:
from instanovo.transformer.model import InstaNovo

from tqdm import tqdm
import torch
import os
import numpy as np
import pandas as pd
import urllib.request
import zipfile

We can download the model checkpoint directly from the [InstaNovo releases](https://github.com/instadeepai/InstaNovo/releases).

In [ ]:
# Download checkpoint locally
os.makedirs('checkpoints', exist_ok=True)
url = "https://github.com/instadeepai/InstaNovo/releases/download/0.1.4/instanovo_yeast.pt"
file_path = os.path.join('checkpoints', 'instanovo_yeast.pt')
if not os.path.exists(file_path):
    urllib.request.urlretrieve(url, file_path)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

Loading the model...

In [ ]:
# model_path = "./checkpoints/instanovo_yeast.pt"
model_path = "../checkpoints/extended_147b2a84/epoch=3-step=800000.ckpt"

In [ ]:
model, config = InstaNovo.load(model_path)
model = model.to(device).eval()

## Loading the nine-species dataset
Download the [yeast test fold of the nine-species dataset](https://huggingface.co/datasets/InstaDeepAI/instanovo_ninespecies_exclude_yeast) dataset from HuggingFace.

We can use our SpectrumDataFrame to download this directly. SpectrumDataFrame is a special data class used by InstaNovo to read and write from multiple formats, including mgf, mzml, mzxml, pandas, polars, HuggingFace, etc.

In [ ]:
from instanovo.utils import SpectrumDataFrame

sdf = SpectrumDataFrame.from_huggingface("InstaDeepAI/ms_ninespecies_benchmark", is_annotated=True, shuffle=False, split="test[:10%]")

In [ ]:
sdf.to_pandas().head(5)

In [ ]:
from instanovo.transformer.dataset import SpectrumDataset, collate_batch

ds = SpectrumDataset(
    sdf, 
    model.residue_set, 
    config.get("n_peaks", 200),
    return_str=True,
    annotated=True,
)

In [ ]:
from torch.utils.data import DataLoader

# When using SpectrumDataFrame, workers and shuffle is handled internally.
dl = DataLoader(ds, batch_size=64, shuffle=False, num_workers=0, collate_fn=collate_batch)

In [ ]:
batch = next(iter(dl))

spectra, precursors, spectra_mask, peptides, _ = batch
spectra = spectra.to(device)
precursors = precursors.to(device)

In [ ]:
spectra.shape, precursors.shape

## Knapsack beam-search decoder

Setup knapsack beam search decoder. This may take a few minutes.

In [ ]:
from instanovo.constants import MASS_SCALE
from instanovo.inference.knapsack import Knapsack
from instanovo.inference.knapsack_beam_search import KnapsackBeamSearchDecoder

num_beams = 5

def _setup_knapsack(model: InstaNovo) -> Knapsack:
    # Cannot allow negative masses in knapsack graph
    if "(-17.03)" in model.residue_set.residue_masses:
        model.residue_set.residue_masses["(-17.03)"] = 1e3
    
    residue_masses = dict(model.residue_set.residue_masses.copy())
    if any([x < 0 for x in residue_masses.values()]):
        raise NotImplementedError(
            "Negative mass found in residues, this will break the knapsack graph. Either disable knapsack or use strictly positive masses"
        )
    for special_residue in list(model.residue_set.residue_to_index.keys())[:3]:
        residue_masses[special_residue] = 0
    residue_indices = model.residue_set.residue_to_index
    return Knapsack.construct_knapsack(
        residue_masses=residue_masses,
        residue_indices=residue_indices,
        max_mass=4000.00,
        mass_scale=MASS_SCALE,
    )

knapsack_path = "./checkpoints/knapsack/"

if not os.path.exists(knapsack_path):
    print("Knapsack path missing or not specified, generating...")
    knapsack = _setup_knapsack(model)
    decoder = KnapsackBeamSearchDecoder(model, knapsack)
    # Optionally save knapsack
    # print(f"Saving knapsack to {knapsack_path}")
    # knapsack.save(knapsack_path)
else:
    print("Knapsack path found. Loading...")
    decoder = KnapsackBeamSearchDecoder.from_file(model=model, path=knapsack_path)

Or use greedy search (fastest) or plain BeamSearch

In [ ]:
from instanovo.inference import GreedyDecoder
from instanovo.inference import BeamSearchDecoder

num_beams = 1

if num_beams > 1:
    decoder = BeamSearchDecoder(model=model)
else:    
    decoder = GreedyDecoder(model=model)

## Inference time 🚀

Evaluating a single batch...

In [ ]:
from instanovo.inference import ScoredSequence

with torch.no_grad():
    p = decoder.decode(
        spectra=spectra,
        precursors=precursors,
        beam_size=num_beams,
        max_length=config["max_length"],
    )

preds = [
    x.sequence 
    if isinstance(x, ScoredSequence) 
    else [] 
    for x in p
]
probs = [
    x.sequence_log_probability
    if isinstance(x, ScoredSequence)
    else -float("inf")
    for x in p
]

In [ ]:
from instanovo.utils.metrics import Metrics

metrics = Metrics(model.residue_set, config["isotope_error_range"])

In [ ]:
aa_precision, aa_recall, peptide_recall, peptide_precision = metrics.compute_precision_recall(peptides, preds)
peptide_recall

Evaluating on the yeast test fold of the nine-species dataset:

In [ ]:
preds = []
targs = []
probs = []

for _, batch in tqdm(enumerate(dl), total=len(dl)):
    spectra, precursors, _, peptides, _ = batch
    spectra = spectra.to(device)
    precursors = precursors.to(device)

    with torch.no_grad():
        p = decoder.decode(
            spectra=spectra,
            precursors=precursors,
            beam_size=config["n_beams"],
            max_length=config["max_length"],
        )

    preds += [
        x.sequence 
        if isinstance(x, ScoredSequence) 
        else [] 
        for x in p
    ]
    probs += [
        x.sequence_log_probability
        if isinstance(x, ScoredSequence)
        else -float("inf")
        for x in p
    ]
    targs += list(peptides)

In [ ]:
aa_precision, aa_recall, peptide_recall, peptide_precision = metrics.compute_precision_recall(targs, preds)
aa_error_rate = metrics.compute_aa_er(targs, preds)
auc = metrics.calc_auc(targs, preds, np.exp(pd.Series(probs)))

print(f"amino acid error rate:    {aa_error_rate:.5f}")
print(f"amino acid precision:     {aa_precision:.5f}")
print(f"amino acid recall:        {aa_recall:.5f}")
print(f"peptide precision:        {peptide_precision:.5f}")
print(f"peptide recall:           {peptide_recall:.5f}")
print(f"area under the PR curve:  {auc:.5f}")

_Note: to reproduce the results of the paper, the entire Yeast test set should be evaluated with the 0.1.7 release of InstaNovo._

### Saving the predictions...

In [ ]:
pred_df = pd.DataFrame({
    "targets": targs,
    "predictions": ["".join(x) for x in preds],
    "log_probabilities": probs,
})
pred_df.head()

In [ ]:
pred_df.to_csv("predictions.csv", index=False)

## InstaNovo+: Iterative Refinement with a Diffusion Model [OUTDATED]
<font color="red">**This code is outdated with the 1.0 release of InstaNovo, please use [release 0.7.1](https://github.com/instadeepai/InstaNovo/releases/tag/0.1.7)**</font>

In this section, we show how to refine the predictions from the transformer model with a diffusion model.

First, we download the model checkpoint.

In [ ]:
url = "https://github.com/instadeepai/InstaNovo/releases/download/0.1.5/instanovoplus_yeast.zip"
checkpoint_dir = "./checkpoints/"
zip_file_path = os.path.join(checkpoint_dir, "instanovoplus_yeast.zip")

# Download the file only if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)
if not os.path.exists(zip_file_path):
    urllib.request.urlretrieve(url, zip_file_path)
    print(f"Downloaded: {zip_file_path}")
else:
    print(f"File already exists: {zip_file_path}")

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(checkpoint_dir)
    print(f"Extracted to: {checkpoint_dir}")

Next, we load the checkpoint and create a decoder object.

In [ ]:
from instanovo.diffusion.multinomial_diffusion import MultinomialDiffusion
from instanovo.inference.diffusion import DiffusionDecoder

diffusion_model = MultinomialDiffusion.load("./checkpoints/diffusion_checkpoint")
diffusion_model = diffusion_model.to(device).eval()
diffusion_decoder = DiffusionDecoder(model=diffusion_model)


Then we prepare the inference data loader using predictions from the InstaNovo transformer model.

In [ ]:
import pandas as pd
import polars as pl
from instanovo.diffusion.dataset import AnnotatedPolarsSpectrumDataset
from instanovo.diffusion.dataset import collate_batches

diffusion_dataset = AnnotatedPolarsSpectrumDataset(
    pl.from_pandas(pd.DataFrame(dataset)), peptides=preds
)

diffusion_data_loader = DataLoader(diffusion_dataset, batch_size=64, shuffle=False,
                                   collate_fn=collate_batches(
                                       residues=diffusion_model.residues,
                                       max_length=diffusion_model.config.max_length,
                                       time_steps=diffusion_decoder.time_steps,
                                       annotated=True
                                   ))

Finally, we predict sequences by iterating over the spectra and refining the InstaNovo predictions.

In [ ]:
predictions = []
log_probs = []

for batch in tqdm(diffusion_data_loader, total=len(diffusion_data_loader)):
    spectra, spectra_padding_mask, precursors, peptides, peptide_padding_mask = batch
    spectra = spectra.to(device)
    spectra_padding_mask = spectra_padding_mask.to(device)
    precursors = precursors.to(device)
    peptides = peptides.to(device)
    peptide_padding_mask = peptide_padding_mask.to(device)

    with torch.no_grad():
        batch_predictions, batch_log_probs = diffusion_decoder.decode(
            spectra=spectra,
            spectra_padding_mask=spectra_padding_mask,
            precursors=precursors,
            initial_sequence=peptides
        )
    predictions.extend(["".join(sequence) for sequence in batch_predictions])
    log_probs.extend(batch_log_probs)

Iterative refinement improves performance on this sample of the Nine Species dataset. (To replicate the performance reported in the paper, you would need to evaluate on the entire dataset.) 

In [ ]:
aa_precision_refined, aa_recall_refined, peptide_recall_refined, peptide_precision_refined = metrics.compute_precision_recall(targs, predictions=predictions)
aa_error_rate_refined = metrics.compute_aa_er(targs, predictions)
auc_refined = metrics.calc_auc(targs, predictions, np.exp(pd.Series(log_probs)))

print(f"amino acid error rate:   {aa_error_rate_refined}")
print(f"amino acid precision:  {aa_precision_refined}")
print(f"amino acid recall:  {aa_recall_refined}")
print(f"peptide precision:  {peptide_precision_refined}")
print(f"peptide recall:  {peptide_recall_refined}")
print(f"area under the ROC curve:   {auc_refined}")

In [ ]:
print(f"Decrease in AA error rate: {100*(aa_error_rate - aa_error_rate_refined):.2f}%")
print(f"Increase in AA precision: {100*(aa_precision_refined - aa_precision):.2f}%")
print(f"Increase in AA recall: {100*(aa_recall_refined - aa_recall):.2f}%")
print(f"Increase in peptide precision: {100*(peptide_precision_refined - peptide_precision):.2f}%")
print(f"Increase in peptide recall: {100*(peptide_recall_refined - peptide_recall):.2f}%")
print(f"Increase in AUC: {100*(auc_refined - auc):.2f}%")

In [ ]:
diffusion_predictions = pd.DataFrame({
    "targets": targs,
    "predictions": predictions,
    "log_probabilities": log_probs,
})
diffusion_predictions.head()

In [ ]:
diffusion_predictions.to_csv("diffusion_predictions.csv", index=False)